# Домашнее задание №4: обработка текстов, нейронные сети на keras

<span style="color: red; font-size: 14pt">Дедлайн: 13 сентября 23:59</span>

**Оформление дз**: 

- Task short name: ``NN_NLP``.
- Выполненное дз сохраните в файл ``ML2018_<фамилия>_HW#.ipynb``, к примеру -- ``ML2018_ivanov_HW4.ipynb``
- Присылайте выполненное задание на почту `` mailto:ml4megafon_2018_08@bigdatateam.org `` с темой письма `` HW# Short name. ФИО ``. 

    Например: `` HW4 NN_NLP. Иванов Иван Иванович. ``

**Вопросы**:
- Свои вопросы присылайте в Telegram.

**Фидбек**:
- Пожалуйста, оставьте свой отзыв после выполнения домашнего задания по сссылке:

    http://bit.ly/ml4megafon_august18_hw4keras_feedback

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from functools import reduce

from sklearn.datasets import fetch_20newsgroups

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score

### Часть 0. Векторизация текстов, повторение.

#### Bag of Words (CountVectorizer)

Для начала воспользуемся подходом мешка слов (`bag of words`). Он создает вектор длиной в количество уникальных слов во всех текстах, подсчитывает количество вхождений каждого слова в каждый текст и подставляет это число на соответствующую позицию в векторе. Данный метод доступен в модуле `sklearn.feature_extraction.text` в классе `CountVectorizer`. Но столь простой метод можно реализовать и самостоятельно. Для Вашего удобства код, реализующий данный функционал, приведен ниже.

In [ ]:
texts = [['i', 'have', 'a', 'cat'],
         ['he', 'have', 'a', 'dog'], # Не обращайте внимания на грамматику, считаем слова приведенными в начальную форму
         ['he', 'and', 'i', 'have', 'a', 'cat', 'and', 'a', 'dog'],
         ['i', 'have', 'a', 'pencil']
        ]

dictionary = list(enumerate(set(reduce(lambda x, y: x + y, texts))))

def vectorize(text):
    vector = np.zeros(len(dictionary))
    for i, word in dictionary:
        num = 0
        for w in text:
            if w == word:
                num += 1
        if num:
            vector[i] = num
    return vector

for t in texts:
    print(vectorize(t))

![img](https://habrastorage.org/files/549/810/b75/549810b757f94e4784b6780d84a1112a.png)

In [ ]:
count_vectorizer = CountVectorizer(min_df=1)
vectorized = count_vectorizer.fit_transform([' '.join(x) for x in texts])

In [ ]:
# Имена признаков (т.е. слов, которые употребляются в тексте)
count_vectorizer.get_feature_names()

In [ ]:
vectorized.toarray()

Обращаем Ваше внимание, что размерность данных векторов стала меньше. Это произошло из-за стандартных настроек `CountVectorizer`'а: по умолчанию рассматриваются только токены длины 2 и выше. Ниже приведена выдержка из документации.
```
token_pattern : string

Regular expression denoting what constitutes a “token”, only used if analyzer == 'word'. The default regexp select tokens of 2 or more alphanumeric characters (punctuation is completely ignored and always treated as a token separator).

```


#### TF-IDF (TfidfVectorizer)

Одним из основных классических механизмов является TF-IDF [вики](https://ru.wikipedia.org/wiki/TF-IDF), который позволяет получать достаточно информативное представление текстов. Его так же можно импортировать из `sklearn.feature_extraction.text `. Ниже приведен пример его вызова и преобразования тех же игрушечных текстов, что использовались выше.

In [ ]:
vectorizer = TfidfVectorizer()
vectorized_texts = vectorizer.fit_transform([' '.join(x) for x in texts])
# Не забывайте, что у vectorizer есть и метод fit_transform, и метод transform
vectorized_texts_without_fitting = vectorizer.transform([' '.join(x) for x in texts])

### Часть 1. Линейные модели (15%)

Будем использовать тексты новостей из датасета `` The 20 newsgroups text dataset ``. По тексту новости требуется определить наиболее вероятную категорию (иначе говоря, тему). Для начала будем работать только с двумя категориями. Обучите линейный классификатор для разделения двух классов. Данные разделены на train и test с помощью `train_test_split`. Обратите внимание на исходный формат: тексты доступны в поле `.data`, метки классов в поле `.target`.

Сначала рассмотрим категории `easy_categories`, которые сильно разнятся "на глаз": огнестрельное оружие и научные заметки о космосе.
(Затем проделайте все те же действия для другой пары тематик `hard_categories`: автомобильные новости о автомобилях и мотоциклах соответственно).

In [ ]:
easy_categories = ['talk.politics.guns', 'sci.space']
hard_categories = ['rec.motorcycles', 'rec.autos']

In [ ]:
two_groups_data = fetch_20newsgroups(subset='all', 
                                     categories=easy_categories,
                                     remove=('headers', 'footers', 'quotes'))

In [ ]:
train_texts, test_texts, train_targets, test_targets = train_test_split(two_groups_data.data, 
                                                                        two_groups_data.target, 
                                                                        test_size=0.33)

Преобразуем тексты в разреженные векторы с помощью `CountVectorizer`. 

In [ ]:
vectorizer = CountVectorizer()# TfidfVectorizer() 
data_train = vectorizer.fit_transform(train_texts)
# Не забывайте, что у vectorizer есть и метод fit_transform, и метод transform
data_test = vectorizer.transform(test_texts)

Обучите логистическую регрессию. Оцените качество классификации на отложенной выборке с помощью `accuracy` и `f1_score` ([Wikipedia](https://en.wikipedia.org/wiki/F1_score)).

In [ ]:
lr = LogisticRegression()

In [ ]:
# Ваш код здесь

Сравните качество классификации (по метрике `accuracy`) на первой и второй паре тематик (`easy_categories` и `hard_categories` соответственно).

In [ ]:
# Ваш код здесь

Проделайте аналогичные манипуляции (векторизация текстов -> обучение модели -> предсказание на тестовой выборке -> оценка результатов с помощью метрики `accuracy_score`) используя `TfidfVectorizer` для векторизации. Стали ли результаты лучше?

In [ ]:
# Ваш код здесь

### Часть 2. Random Forest (20%)

Теперь будем работать со всеми 20 категориями до определенной даты (`subset='train'`). Разделите выборку на обучающую и валидационную в отношении 90 на 10 (не забудьте перемешать данные). Переведите тексты в векторное представление с помощью `TfidfVectorizer`. Попробуйте обучить `Random Forest` для решения данной задачи.

In [ ]:
whole_data = fetch_20newsgroups(subset='train', 
                                remove=('headers', 'footers', 'quotes'))
texts = whole_data.data
labels = whole_data.target
labels_index = {name: idx 
                for idx, name in enumerate(whole_data.target_names)}  

Будем использовать для векторизации `TfidfVectorizer`.

In [ ]:
vectorizer = TfidfVectorizer()
vectorized_texts = vectorizer.fit_transform()# Your code here

In [ ]:
# Your code for train-val split here

In [ ]:
from sklearn.ensemble import RandomForestClassifier
random_forest_clf = RandomForestClassifier()

# Ваш код здесь

Покажите, как менялись значения критерия качества `accuracy` на валидационной выборке при увеличении параметра `n_estimators`. Видны ли следы переобучения?

In [ ]:
# Ваш код здесь

### Часть 3. Embeddings & Neural Networks (55%)

Обратимся к нейронным сетям для решения данной задачи.

In [ ]:
import os
import keras

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Model
from keras.initializers import Constant

Теперь воспользуемся предобученным представлением для слов GloVe (подробнее [здесь](https://nlp.stanford.edu/projects/glove/)). В переменной `GLOVE_DIR` укажите директорию, в которой находится предобученное представление GloVe.

Будем работать с последовательностями не длиннее `MAX_SEQUENCE_LENGTH`.

In [ ]:
GLOVE_DIR = ''
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 100

embeddings_index = {}
with open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')) as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Загружено %s представлений слов.' % len(embeddings_index))


Воспользуемся классом `Tokenizer` из `keras.preprocessing.text` для первоначального кодирования слов. Приведем строки к единой длинне используя паддинг с помощью `pad_sequences` из `keras.preprocessing.sequence`.

In [ ]:
# vectorize the text samples into a 2D integer tensor
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Данные содержат %s уникальных токенов.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Разделите выборку на обучающую (`x_train, y_train`) и валидационную (`x_val, y_val`) в соотношении 90 на 10. Валидационная выборка понадобится для оценки качества классификации в процессе обучения.

In [ ]:
### Your code here

Теперь воспользуемся GloVe для кодирования слов. Для этого сформируем `embedding_matrix` и подадим ее в качестве инициализационной константы `Embedding` слою из `keras.layers`.

In [ ]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in word_index.items():
    if i >= MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = keras.layers.Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

Ниже приведен простой вариант сети. Улучшите ее и получите хотя бы 60% Accuracy на валидационной выборке. 

In [ ]:
# Creating a 1D convnet with global maxpooling
sequence_input = keras.layers.Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
# Your architecture here
x = keras.layers.GlobalMaxPooling1D()(embedded_sequences)
x = keras.layers.Dense(32, activation='relu')(x)
preds = keras.layers.Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='adagrad',
              metrics=['acc'])


In [ ]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;


Для улучшения результатов можно добавить слоев (например `Conv1D` и `MaxPooling1D`), поменять оптимизатор или сделать что-то еще :)

В ходе обучения модель может начать расходиться (иногда так случается), так что рекомендуем сохранять версию сети, показавшую лучший результат на данный момент (это можно сделать с помощью `model.save_weights`). Загрузить модель можно с помощью `model.load_weights`.

### Часть 4. Проверка модели в *реальных* условиях (10%)

В реальной жизни новые данные становятся доступны с течением времени. Проверьте качество лучшей нейросети и Random Forest'а на отложенной выборке. В ней содержатся данные, полученные после определенной даты. На обучающей выборке использовались лишь данные, доступные до этой даты.

In [ ]:
delayed_test_data = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))
delayed_texts = whole_data.data
delayed_labels = whole_data.target
delayed_labels_index = {name: idx 
                for idx, name in enumerate(delayed_test_data.target_names)}  

Обращаем Ваше внимание, приводить текстовую информацию в векторный вид нужно тем же `transformer`'ом, что использовался при обучении/валидации.

In [ ]:
# Ваш код здесь

Пожалуйста, оставьте отзыв о домашнем задании: [link](http://bit.ly/ml4megafon_august18_hw4keras_feedback)